In [117]:
# Load SentiLex-flex-PT02 dictionary
import pandas as pd
import re

file = open('C:\\Users\\PICHAU\\Documents\\ArthurAnzai\\lexiconPT\\data-raw\\SentiLex-lem-PT02.txt', 'r', encoding='utf-8')

lines = []

for line in file:
    lines.append(line.strip('.'))
    
file.close()

data = {'word':[],'PoS': [],  'polarity_target': [], 'polarity': [], 'polarity_classification': []}

for line in lines:
    columns = re.split(r'[.;]', line)
    data['word'].append(columns[0])
    data['PoS'].append(columns[1].split('=')[1])
    #data['FLEX'].append(columns[2].split('=')[1])
    data['polarity_target'].append(columns[2].split('=')[1])
    data['polarity'].append(columns[3].split('=')[1])
    data['polarity_classification'].append(columns[4].split('=')[1])
    
df = pd.DataFrame(data)

df['polarity'] = pd.to_numeric(df['polarity'])

senti_lex = df

In [149]:
import nltk
import PyPDF2
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from textblob import TextBlob
import re

# Load Lexico v2.0 dictionary
lexico = pd.read_csv('C:\\Users\\PICHAU\\Documents\\ArthurAnzai\\lexiconPT\\data-raw\\lexico_v2.1txt', sep=',', header=None)
lexico.columns = ['word', 'polarity', 'sentiment']

# Load OPLexico v3.0 dictionary
oplexico = pd.read_csv('C:\\Users\\PICHAU\\Documents\\ArthurAnzai\\lexiconPT\\data-raw\\oplexicon_v3.0\\lexico_v3.0.txt', sep=',', header=None)
oplexico.columns = ['word', 'polarity', 'sentiment','classification']

# Load SentiLex-flex-PT02 dictionary

file = open('C:\\Users\\PICHAU\\Documents\\ArthurAnzai\\lexiconPT\\data-raw\\SentiLex-lem-PT02.txt', 'r', encoding='utf-8')
lines = []
for line in file:
    lines.append(line.strip('.'))  
file.close()
data = {'word':[],'PoS': [],  'polarity_target': [], 'polarity': [], 'polarity_classification': []}
for line in lines:
    columns = re.split(r'[.;]', line)
    data['word'].append(columns[0])
    data['PoS'].append(columns[1].split('=')[1])
    #data['FLEX'].append(columns[2].split('=')[1])
    data['polarity_target'].append(columns[2].split('=')[1])
    data['polarity'].append(columns[3].split('=')[1])
    data['polarity_classification'].append(columns[4].split('=')[1])
   
df = pd.DataFrame(data)
df['polarity'] = pd.to_numeric(df['polarity'])
senti_lex = df

# Load Vader sentiment analyzer
vader = SentimentIntensityAnalyzer()

# Read in PDF file
pdf_file = open('Release de Resultados 1T22.pdf', 'rb')
pdf_reader = PyPDF2.PdfFileReader(pdf_file)

# Tokenize PDF text into sentences

text = ''
for i in range(pdf_reader.getNumPages()):
    page = pdf_reader.getPage(i)
    text += page.extractText()
sentences = sent_tokenize(text)

# Define sentiment score functions
def lexico_score(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    score = 0
    for token in tokens:
        if token in lexico['word'].values:
            score += lexico.loc[lexico['word'] == token, 'sentiment'].values[0]
    return score


#def oplexicon_score(sentence):
#    oplexicon_score = 0
#    for word in sentence.split():
#        if word.lower() in oplexico['word'].values:
#            oplexicon_score += oplexico.loc[(oplexico['word'] == word.lower()), 'sentiment'].values[0]
#    return oplexicon_score   


def oplexicon_score(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    score = 0
    for token in tokens:
        if token in oplexico['word'].values:
            score += oplexico.loc[oplexico['word'] == token, 'sentiment'].values[0]
    return score

def sentilex_score(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    score = 0
    for token in tokens:
        if token in senti_lex['word'].values:
            score += senti_lex.loc[senti_lex['word'] == token, 'polarity'].values[0]
    return score

# Compute sentiment scores for each sentence
lexico_scores = [lexico_score(sentence) for sentence in sentences]
sentilex_scores = [sentilex_score(sentence) for sentence in sentences]
oplexicon_score = [oplexicon_score(sentence) for sentence in sentences]
vader_scores = [vader.polarity_scores(sentence)['compound'] for sentence in sentences]
textblob_scores = [TextBlob(sentence).sentiment.polarity for sentence in sentences]


results = []
# Compare the scores
for i in range(len(sentences)):
    result = {
            'sentence': i,
            'vader_score': vader_scores[i],
            'lexico_score': lexico_scores[i],
            'sentilex_score': sentilex_scores[i],
            'oplexicon_score': oplexicon_score[i],
            'textblob_score': textblob_scores[i]
        }
    results.append(result)
    
# Compute total sentiment scores for the document
lexico_total = sum(lexico_scores)
sentilex_total = sum(sentilex_scores)
oplexicon_total = sum(oplexicon_score)
vader_total = sum(vader_scores)
textblob_total = sum(textblob_scores)

# Compute average sentiment scores for the document
num_sentences = len(sentences)
lexico_avg = lexico_total / num_sentences
sentilex_avg = sentilex_total / num_sentences
oplexicon_avg = oplexicon_total / num_sentences
vader_avg = vader_total / num_sentences
textblob_avg = textblob_total / num_sentences

# Print the final results
print("Final Results:")
print(f"Lexico Avg Score: {lexico_avg}")
print(f"Sentilex Avg Score: {sentilex_avg}")
print(f"OPLexicon Avg Score: {oplexicon_avg}")
print(f"Vader Avg Score: {vader_avg}")
print(f"TextBlob Avg Score: {textblob_avg}")



Final Results:
Lexico Avg Score: -0.2846153846153846
Sentilex Avg Score: 0.13846153846153847
OPLexicon Avg Score: -0.36153846153846153
Vader Avg Score: -0.09990384615384612
TextBlob Avg Score: 0.024532967032967024


In [151]:
pdf_file = open('Release de Resultados 1T22.pdf', 'rb')
pdf_reader = PyPDF2.PdfFileReader(pdf_file)

# Tokenize PDF text into sentences
text = ''
for i in range(pdf_reader.getNumPages()):
    page = pdf_reader.getPage(i)
    text += page.extractText()
sentences = sent_tokenize(text)

# Define sentiment score functions
def lexico_score(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    score = 0
    for token in tokens:
        if token in lexico['word'].values:
            score += lexico.loc[lexico['word'] == token, 'sentiment'].values[0]
    return score

def oplexicon_score(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    score = 0
    for token in tokens:
        if token in oplexico['word'].values:
            score += oplexico.loc[oplexico['word'] == token, 'sentiment'].values[0]
    return score

def sentilex_score(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    score = 0
    for token in tokens:
        if token in senti_lex['word'].values:
            score += senti_lex.loc[senti_lex['word'] == token, 'polarity'].values[0]
    return score

def vader_score(sentence):
    return vader.polarity_scores(sentence)['compound']

def textblob_score(sentence):
    return TextBlob(sentence).sentiment.polarity


# Compute sentiment scores for each sentence
results = []
for i, sentence in enumerate(sentences):
    # First, try to use the lexico_score
    score = lexico_score(sentence)
    source = 'lexico_score'
    
    # If the score is 0, try to use the oplexicon_score
    if score == 0:
        score = oplexicon_score(sentence)
        source = 'oplexicon_score'
        
    # If the score is 0, try to use the sentilex_score
    if score == 0:
        score = sentilex_score(sentence)
        source = 'sentilex_score'
    
    # If the score is 0, use the Vader score
    if score == 0:
        score = vader.polarity_scores(sentence)['compound']
        source = 'vader_score'
    
    # If the score is still 0, use the TextBlob score
    if score == 0:
        score = TextBlob(sentence).sentiment.polarity
        source = 'textblob_score'
    
    result = {
        'sentence': i,
        'score': score,
        'source': source,
        'text': sentence,
    }
    
    results.append(result)
    
# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)
results_df

# Compute overall sentiment score
overall_score = results_df['score'].mean()
overall_score


-0.3111184615384615

In [164]:
import nltk
import PyPDF2
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from textblob import TextBlob
import re

# Load Lexico v2.0 dictionary
lexico = pd.read_csv('C:\\Users\\PICHAU\\Documents\\ArthurAnzai\\lexiconPT\\data-raw\\lexico_v2.1txt', sep=',', header=None)
lexico.columns = ['word', 'polarity', 'sentiment']

# Load OPLexico v3.0 dictionary
oplexico = pd.read_csv('C:\\Users\\PICHAU\\Documents\\ArthurAnzai\\lexiconPT\\data-raw\\oplexicon_v3.0\\lexico_v3.0.txt', sep=',', header=None)
oplexico.columns = ['word', 'polarity', 'sentiment','classification']

# Load SentiLex-flex-PT02 dictionary

file = open('C:\\Users\\PICHAU\\Documents\\ArthurAnzai\\lexiconPT\\data-raw\\SentiLex-lem-PT02.txt', 'r', encoding='utf-8')

lines = []

for line in file:
    lines.append(line.strip('.'))  
file.close()
data = {'word':[],'PoS': [],  'polarity_target': [], 'polarity': [], 'polarity_classification': []}
for line in lines:
    columns = re.split(r'[.;]', line)
    data['word'].append(columns[0])
    data['PoS'].append(columns[1].split('=')[1])
    #data['FLEX'].append(columns[2].split('=')[1])
    data['polarity_target'].append(columns[2].split('=')[1])
    data['polarity'].append(columns[3].split('=')[1])
    data['polarity_classification'].append(columns[4].split('=')[1])
   
df = pd.DataFrame(data)
df['polarity'] = pd.to_numeric(df['polarity'])
senti_lex = df

# Load Vader sentiment analyzer
vader = SentimentIntensityAnalyzer()

# Define sentiment score functions
def lexico_score(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    score = 0
    for token in tokens:
        if token in lexico['word'].values:
            score += lexico.loc[lexico['word'] == token, 'sentiment'].values[0]
    return score


def oplexicon_score(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    score = 0
    for token in tokens:
        if token in oplexico['word'].values:
            score += oplexico.loc[oplexico['word'] == token, 'sentiment'].values[0]
    return score


def sentilex_score(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    score = 0
    for token in tokens:
        if token in senti_lex['word'].values:
            score += senti_lex.loc[senti_lex['word'] == token, 'polarity'].values[0]
    return score

def sentiment_analysis_sentence(pdf):
    # Read in PDF file
    pdf_file = open(pdf, 'rb')
    pdf_reader = PyPDF2.PdfFileReader(pdf_file)

    # Tokenize PDF text into sentences

    text = ''
    for i in range(pdf_reader.getNumPages()):
        page = pdf_reader.getPage(i)
        text += page.extractText()
    sentences = sent_tokenize(text)

    # Compute sentiment scores for each sentence
    lexico_scores = [lexico_score(sentence) for sentence in sentences]
    sentilex_scores = [sentilex_score(sentence) for sentence in sentences]
    oplexicon_scores = [oplexicon_score(sentence) for sentence in sentences]
    vader_scores = [vader.polarity_scores(sentence)['compound'] for sentence in sentences]
    textblob_scores = [TextBlob(sentence).sentiment.polarity for sentence in sentences]


    results = []
    # Compare the scores
    for i in range(len(sentences)):
        result = {
                'sentence': i,
                'vader_score': vader_scores[i],
                'lexico_score': lexico_scores[i],
                'sentilex_score': sentilex_scores[i],
                'oplexicon_score': oplexicon_scores[i],
                'textblob_score': textblob_scores[i]
            }
        results.append(result)

    # Compute total sentiment scores for the document
    lexico_total = sum(lexico_scores)
    sentilex_total = sum(sentilex_scores)
    oplexicon_total = sum(oplexicon_scores)
    vader_total = sum(vader_scores)
    textblob_total = sum(textblob_scores)

    # Compute average sentiment scores for the document
    num_sentences = len(sentences)
    lexico_avg = lexico_total / num_sentences
    sentilex_avg = sentilex_total / num_sentences
    oplexicon_avg = oplexicon_total / num_sentences
    vader_avg = vader_total / num_sentences
    textblob_avg = textblob_total / num_sentences
    
    # Print the final results
    print("Final Results:")
    print(f"Lexico Avg Score: {lexico_avg}")
    print(f"Sentilex Avg Score: {sentilex_avg}")
    print(f"OPLexicon Avg Score: {oplexicon_avg}")
    print(f"Vader Avg Score: {vader_avg}")
    print(f"TextBlob Avg Score: {textblob_avg}")



Final Results:
Lexico Avg Score: -0.2846153846153846
Sentilex Avg Score: 0.13846153846153847
OPLexicon Avg Score: -0.36153846153846153
Vader Avg Score: -0.09990384615384612
TextBlob Avg Score: 0.024532967032967024


In [165]:

def sentiment_analysis_overall(pdf):
    
    pdf_file = open('Release de Resultados 1T22.pdf', 'rb')
    pdf_reader = PyPDF2.PdfFileReader(pdf_file)

    # Tokenize PDF text into sentences
    text = ''
    for i in range(pdf_reader.getNumPages()):
        page = pdf_reader.getPage(i)
        text += page.extractText()
    sentences = sent_tokenize(text)
    
    # Compute sentiment scores for each sentence
    results = []
    for i, sentence in enumerate(sentences):
        # First, try to use the lexico_score
        score = lexico_score(sentence)
        source = 'lexico_score'

        # If the score is 0, try to use the oplexicon_score
        if score == 0:
            score = oplexicon_score(sentence)
            source = 'oplexicon_score'

        # If the score is 0, try to use the sentilex_score
        if score == 0:
            score = sentilex_score(sentence)
            source = 'sentilex_score'

        # If the score is 0, use the Vader score
        if score == 0:
            score = vader.polarity_scores(sentence)['compound']
            source = 'vader_score'

        # If the score is still 0, use the TextBlob score
        if score == 0:
            score = TextBlob(sentence).sentiment.polarity
            source = 'textblob_score'

        result = {
            'sentence': i,
            'score': score,
            'source': source,
            'text': sentence,
        }

        results.append(result)

    # Convert results to a pandas DataFrame
    results_df = pd.DataFrame(results)
    results_df

    # Compute overall sentiment score
    overall_score = results_df['score'].mean()
    overall_score
    print(f"Overall Sentiment Score: {overall_score}")


In [167]:
sentiment_analysis_sentence('Release de Resultados 1T22.pdf')
sentiment_analysis_overall('Release de Resultados 1T22.pdf')


Final Results:
Lexico Avg Score: -0.2846153846153846
Sentilex Avg Score: 0.13846153846153847
OPLexicon Avg Score: -0.36153846153846153
Vader Avg Score: -0.09990384615384612
TextBlob Avg Score: 0.024532967032967024
Overall Sentiment Score: -0.3111184615384615
